In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import csc_matrix,find,coo_matrix
import torch
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import os
import time

In [ ]:
### Variables ###

# the path of raw rule files generated by AMIE 3
data_path_list = [
    './rules_file/rules_0.txt',
    './rules_file/rules_1.txt',
    './rules_file/rules_2.txt',
    './rules_file/rules_3.txt',
    './rules_file/rules_4.txt'
]

# path
train_hrt_data_path = './ogb/wikikg90m_kddcup2021/processed/train_hrt.npy'
relation_matrix_path = './rel_ht_spmat/'

# the attribute of rule list
V_Rule = 0
V_Head_Coverage = 1
V_Std_Confidence =2
V_PCA_Confidence = 3
V_Positive_Examples = 4
V_Body_size = 5
V_PCA_Body_size = 6
V_Functional_variable = 7

# other variables
INV_NUM = 10000

In [ ]:
### Functions about rules ###

# get rules from the raw files
def get_rules_from_file(rule_file_path):
    fp = open(rule_file_path)
    rule_num = -1
    rules_lists = []
    for line in fp.readlines():
        if rule_num == -1:
            items = line.strip().split('\t')
        else:
            rule_list = line.strip().split('\t')
            rule_list[1] = float(rule_list[1]) # Head Coverage
            rule_list[2] = float(rule_list[2]) # Std Confidence
            rule_list[3] = float(rule_list[3]) # PCA Confidence
            rule_list[4] = int(rule_list[4]) # Positive Examples
            rule_list[5] = int(rule_list[5]) # Body size
            rule_list[6] = int(rule_list[6]) # PCA Body size

            rules_lists.append(rule_list)
        rule_num += 1
        
    for i in range(rule_num):
        rules_lists[i][0] = parse_rules(rules_lists[i][0])
    return rules_lists


# get the number of positive and negative examples
def get_rule_PCA_examples(rules_lists):
    rules_dict = {}
    for i in range(len(rules_lists)):
        pos_example = rules_lists[i][V_Positive_Examples]
        neg_example = round( (pos_example / rules_lists[i][V_PCA_Confidence]) - pos_example )
        rule = rules_lists[i][V_Rule]
        if rule not in rules_dict.keys():
            rules_dict[rule] = {}
            rules_dict[rule]['NEG'] = neg_example
            rules_dict[rule]['POS'] = pos_example
        else:
            rules_dict[rule]['NEG'] += neg_example
            rules_dict[rule]['POS'] += pos_example
    return rules_dict


# convert dict to list 
# the element of rules_dict is in the form: { RULE: {NEG，NEG} }
# the element of rules_lists is in the form: [RULE, POS, NEG, PAC_CONFIDENCE]
def get_rule_list_from_dict(rules_dict):
    rules_lists = []
    
    for rule in rules_dict.keys():
        pos_example = rules_dict[rule]['POS']
        neg_example = rules_dict[rule]['NEG']
        pca_confidence = pos_example / (pos_example + neg_example)
        rules_lists.append([rule, pos_example, neg_example, pca_confidence])
    return rules_lists


# parse the raw rule files
def parse_rules(rule_str):
    
    rule_str_list = rule_str.split()
    equal_index = rule_str_list.index('=>')
    if equal_index // 3 == 1:
        body_s1 = rule_str_list[0]
        body_s2 = rule_str_list[2]
        body_rel = int(rule_str_list[1])
        
        head_s1 = rule_str_list[4]
        head_s2 = rule_str_list[6]
        head_rel = int(rule_str_list[5])
        
        if head_s1 == body_s1 and head_s2 == body_s2:
            return (body_rel, head_rel)
        elif head_s1 == body_s2 and head_s2 == body_s1:
            return (body_rel + INV_NUM, head_rel)
        else:
            return None
    elif equal_index // 3 == 2:
        body1_s1 = rule_str_list[0]
        body1_s2 = rule_str_list[2]
        body1_rel = int(rule_str_list[1])
        
        body2_s1 = rule_str_list[3]
        body2_s2 = rule_str_list[5]
        body2_rel = int(rule_str_list[4])
        
        head_s1 = rule_str_list[7]
        head_s2 = rule_str_list[9]
        head_rel = int(rule_str_list[8])
        
        if body1_s2 == body2_s1: 
            if body1_s1 == head_s1 and body2_s2 == head_s2: # (a r_1 b) (b r_2 c) => (a r_3 c) 
                return (body1_rel, body2_rel, head_rel) # (a r_1 b) (b r_2 c) => (a r_3 c) 
            elif body1_s1 == head_s2 and body2_s2 == head_s1: # (c r_1 b) (b r_2 a) => (a r_3 c) 
                return (body2_rel + INV_NUM, body1_rel+ INV_NUM, head_rel)  
            else:
                return None
        elif body1_s1 == body2_s1: 
            if body1_s2 == head_s1 and body2_s2 == head_s2: # (a r_1 b) (a r_2 c) => (b r_3 c) 
                return (body1_rel + INV_NUM, body2_rel, head_rel) 
            elif body1_s2 == head_s2 and body2_s2 == head_s1: # (a r_1 b) (a r_2 c) => (c r_3 b) 
                return (body2_rel + INV_NUM, body1_rel, head_rel)  
            else:
                return None
        elif body1_s1 == body2_s2: 
            if body1_s2 == head_s1 and body2_s1 == head_s2: # (a r_1 b) (c r_2 a) => (b r_3 c) 
                return (body1_rel + INV_NUM, body2_rel + INV_NUM, head_rel) 
            elif body1_s2 == head_s2 and body2_s1 == head_s1: # (a r_1 c) (b r_2 a) => (b r_3 c) 
                return (body2_rel, body1_rel, head_rel)  
            else:
                return None
        elif body1_s2 == body2_s2: 
            if body1_s1 == head_s1 and body2_s1 == head_s2: # (b r_1 a) (c r_2 a) => (b r_3 c) 
                return (body1_rel, body2_rel + INV_NUM, head_rel) 
            elif body1_s1 == head_s2 and body2_s1 == head_s1: # (b r_1 a) (c r_2 a) => (c r_3 b) 
                return (body2_rel, body1_rel + INV_NUM, head_rel)  
            else:
                return None
    else:
        return None

# load the relation matrix by id
def load_spmat_by_id(rel):
    return sparse.load_npz(relation_matrix_path + 'rel_ht_spmat_'+str(rel%INV_NUM)+'.npz')

# make predictions by using rules，input format: (r_1, r_2, r_head) or (r_1, r_head)
def get_new_triples_by_rule_scipy(rule):
    if len(rule) == 2:
        r1 = rule[0]
        rh = rule[1]
        
        sp_r1 = load_spmat_by_id(r1)
        sp_rh = load_spmat_by_id(rh)
        
        # for the inverse relations
        if r1 >= INV_NUM:
            sp_r1 = sp_r1.T
        
        sp_rule_head = sp_r1
        sp_new_head = sp_rule_head - coo_matrix.multiply(sp_rule_head, sp_rh)
        sp_new_head = sp_new_head.tocoo()
        
        new_triples = []
        for i in range(len(sp_new_head.row)):
            new_triples.append((sp_new_head.row[i], rh, sp_new_head.col[i]) )
            
        return new_triples
    elif len(rule) == 3:
        r1 = rule[0]
        r2 = rule[1]
        rh = rule[2]
        
        sp_r1 = load_spmat_by_id(r1)
        sp_r2 = load_spmat_by_id(r2)
        sp_rh = load_spmat_by_id(rh)
        
        # for the inverse relations
        if r1 >= INV_NUM:
            sp_r1 = sp_r1.T
        if r2 >= INV_NUM:
            sp_r2 = sp_r2.T
        
        sp_rule_head = sp_r1.dot(sp_r2)
        sp_new_head = sp_rule_head - coo_matrix.multiply(sp_rule_head, sp_rh)
        sp_new_head = sp_new_head.tocoo()
        
        new_triples = []
        for i in range(len(sp_new_head.row)):
            new_triples.append((sp_new_head.row[i], rh, sp_new_head.col[i]))
            
        return new_triples
    else:
        return []

In [ ]:
### Functions about triples ###

# get the set of (h, r) from triples
def get_triples_hr_set(triples):
    hr_set = set()
    for triple in triples:
        hr_set.add((triple[0], triple[1]))
    return hr_set

# get the set of relations from triples
def get_triples_rel_set(triples):
    rel_set = set()
    for triple in triples:
        rel_set.add(triple[1])
    return rel_set

# get rule list concerning relations in rel_set
def rule_filter_rel(rules_lists, rel_set):
    rules_filter = []
    for i in range(len(rules_lists)):
        if rules_lists[i][0][-1] in rel_set:
            rules_filter.append(rules_lists[i])
    return rules_filter

# filter rules by their PCA_Confidence and Positive_Examples
def rule_filter_PCA_POS(rules_lists, pca_conf=0, pos_num=0):
    rules_filter = []
    for i in range(len(rules_lists)):
        if rules_lists[i][-1] >= pca_conf and rules_lists[i][1] >= pos_num:
            rules_filter.append(rules_lists[i])
    return rules_filter

# filter rules by their PCA_Confidence
def rule_filter_PCA_CONF(rules_lists, pca_conf_min=0, pca_conf_max=1.01):
    rules_filter = []
    for i in range(len(rules_lists)):
        if rules_lists[i][-1] >= pca_conf_min and rules_lists[i][-1] < pca_conf_max:
            rules_filter.append(rules_lists[i])
    return rules_filter

# get the dict of r to h
def get_hr_set_dict(hr_set, rel_set):
    hr_set_dict = {}
    for rel in rel_set:
        hr_set_dict[rel] = []
    for hr in hr_set:
        hr_set_dict[hr[1]].append(hr[0])
    return hr_set_dict

# filter the triples concerning relations in rel_set
def filter_triples_by_relset(triples, rel_set):
    filterd_triples = []
    for triple in tqdm(triples):
        if triple[1] in rel_set:
            filterd_triples.append(triple)
    return filterd_triples

# count the frequency of relations in the triples
def count_reltri_num(triples):
    rel_tri_num_dict = {}
    for triple in tqdm(triples):
        if triple[1] in rel_tri_num_dict.keys():
            rel_tri_num_dict[triple[1]] += 1
        else:
            rel_tri_num_dict[triple[1]] = 1
    return rel_tri_num_dict

# filter triples by relations
def filter_triples_by_relset_del(triples, rel_set):
    filterd_triples = []
    for triple in tqdm(triples):
        if triple[1] not in rel_set:
            filterd_triples.append(triple)
    return filterd_triples

In [ ]:
# get the relation matrix #

train_hrt = np.load(train_hrt_data_path) 

num_shape = 87143637 # num_entities
def list_sparse_mat(r_list):
    sp_mat = sparse.coo_matrix((np.ones(len(r_list[0])),(r_list[0],r_list[1])),shape=(num_shape, num_shape))
    return sp_mat

num_relations = 1315

# get the head and tail entities of relations
rel_ht_lists = []
for rel in range(num_relations):
    rel_ht_lists.append([[], []])
    
for i in tqdm(range(len(train_hrt))):
    h = train_hrt[i][0]
    r = train_hrt[i][1]
    t = train_hrt[i][2]
    
    rel_ht_lists[r][0].append(h)
    rel_ht_lists[r][1].append(t)
    
for rel in tqdm(range(num_relations)):
    sp_mat_rel = list_sparse_mat(rel_ht_lists[rel])
    sparse.save_npz(relation_matrix_path + 'rel_ht_spmat_'+str(rel)+'.npz', sp_mat_rel)

In [ ]:
# get all the rules from raw rule files, and merge them
rules_lists_all = []
for data_path in data_path_list:
    rules_lists_all += get_rules_from_file(data_path)
    
rules_dict_all = get_rule_PCA_examples(rules_lists_all)
rules_lists_all_merge = get_rule_list_from_dict(rules_dict_all)

In [ ]:
len(rules_lists_all_merge)

In [ ]:
# delete 'None' value in the rule list
# rules such as（x, x, x）（a, r, b） => (a, r, b) can get 'None' after parsing
del_index = []
len_tmp = len(rules_lists_all_merge)
for i in range(len_tmp):
    if rules_lists_all_merge[i][0] == None:
       del_index.append(i)
for i in range(len(del_index)):
    rules_lists_all_merge.pop(del_index[i]-i)
    
rules_lists_all_merge = sorted(rules_lists_all_merge, key=lambda rule_list: rule_list[-1], reverse=True)

In [ ]:
len(rules_lists_all_merge)

In [ ]:
# get some dicts

num_entities = 87143637

test_hr = np.load('./ogb/wikikg90m_kddcup2021/processed/test_hr.npy').tolist()
val_hr = np.load('./ogb/wikikg90m_kddcup2021/processed/val_hr.npy').tolist()
test_val_hr = test_hr + val_hr

test_val_hr_dict = {}

for itm in test_val_hr:
    if itm[1] not in test_val_hr_dict.keys():
        test_val_hr_dict[itm[1]] = set()
        test_val_hr_dict[itm[1]].add(itm[0])
    else:
        test_val_hr_dict[itm[1]].add(itm[0])

        
# get the set of entities
test_val_ent_set = set()

for itm in test_val_hr:
    test_val_ent_set.add(itm[0])

    
# get the dict of entities
ent_inValTest_dict = {}

for ent in tqdm(range(num_entities)):
    ent_inValTest_dict[ent] = 0
for ent in tqdm(test_val_ent_set):
    ent_inValTest_dict[ent] = 1

In [ ]:
# get the rules with PCA Confidence > 0.95, the process to get the rules with PCA Confidence > 0.99 are similar.

filterd_rules = rule_filter_PCA_CONF(rules_lists_all_merge, 0.95)
filterd_rules_relDict = {}
for rule_itm in filterd_rules:
    rule_head = rule_itm[0][-1]
    if rule_itm[0][-1] in filterd_rules_relDict.keys():
        filterd_rules_relDict[rule_head].append(rule_itm[0])
    else:
        filterd_rules_relDict[rule_head] = [rule_itm[0]]
        
len_rel = []
for key in filterd_rules_relDict.keys():
    len_rel.append(len(filterd_rules_relDict[key]))

In [ ]:
len(filterd_rules)

In [ ]:
# get the predictions 

pca095_path_1 = './pool_files_pca095/'
pca095_path_2 = './pool_files_pca095_filter/'

def rule_task(rel_head, rule_set):
    new_triples = []

    for rule in rule_set:
        new_triples += get_new_triples_by_rule_scipy(rule)
    new_triples = list(set(new_triples))
    
    np.save(pca095_path_1+'pca095_rel_'+str(rel_head)+'_ruleNewTriples.npy', new_triples)
    print(str(rel_head)+' DONE!')

def multi_process_task(max_pool_num):
    process_pool = Pool(max_pool_num)
    for key in filterd_rules_relDict.keys():
        process_pool.apply_async(rule_task, args=(key,filterd_rules_relDict[key]))
    print('Wait the subprocesses ......')
    process_pool.close()
    process_pool.join()
    print('All subprocesses done!')

In [ ]:
multi_process_task(10)

In [ ]:
# keep predictions about valid and test data

def rule_task_inVALTEST_by_ent(rel_head):
    print(str(rel_head)+' START!')
    new_triples = []
    valTest_new_triples = []
    if os.path.exists(pca095_path_2+'pca095_rel_'+str(rel_head)+'_valTestENT_NewTriples.npy'):
        print(str(rel_head)+' ALREADY DONE!')
    else:
        if os.path.exists(pca095_path_1+'pca095_rel_'+str(rel_head)+'_ruleNewTriples.npy'):
            new_triples += np.load(pca095_path_1+'pca095_rel_'+str(rel_head)+'_ruleNewTriples.npy').astype(np.int32).tolist()

        if new_triples != []:
            valTest_new_triples = [ triple for triple in new_triples if ent_inValTest_dict[triple[2]] == 1 or ent_inValTest_dict[triple[0]] == 1 ]
            np.save(pca095_path_2 + '/pca095_rel_'+str(rel_head)+'_valTestENT_NewTriples.npy', np.array(valTest_new_triples).astype(np.int32))

            print(str(rel_head)+' DONE!')
        else:
            print(str(rel_head)+' NONE DONE!')

            
def multi_process_task_inVALTEST_ENT(max_pool_num):
    process_pool = Pool(max_pool_num)
    for rel in filterd_rules_relDict.keys():
        process_pool.apply_async(rule_task_inVALTEST_by_ent, args=(rel,))
    print('Wait the subprocesses ......')
    process_pool.close()
    process_pool.join()
    print('All subprocesses done!')

In [ ]:
multi_process_task_inVALTEST_ENT(10)

In [ ]:
# save the predictions
all_new_triples = []
for rel_head in tqdm(filterd_rules_relDict.keys()):
    if os.path.exists(pca095_path_2+'pca095_rel_'+str(rel_head)+'_valTestENT_NewTriples.npy'):
        all_new_triples += np.load(pca095_path_2+'pca095_rel_'+str(rel_head)+'_valTestENT_NewTriples.npy').tolist()

train_hrt = np.load(train_hrt_data_path)
np.save('./enhanced_triples/train_hrt_pca095_ENT.npy',
        np.vstack((train_hrt, np.array(all_new_triples).astype(np.int32))))